In [6]:
# Automated AI Therapy System - Jupyter Notebook
# This notebook creates a fully automated therapy system using Gemini AI
# It imports and utilizes all existing modules from your therapy system

import os
import sys
import json
import asyncio
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional
import google.generativeai as genai
import re

# Add the therapy system to path
sys.path.append('.')

# Import all your existing therapy system modules
from database import TherapyDatabase, get_or_create_patient, create_session_record, save_assessment_result
from assessment_system import AssessmentSystem
from goal_manager import GoalManager
from homework_system import HomeworkSystem
from therapy_modules import TherapyModuleIntegrator, CBTModule, DBTModule, ACTModule, PsychodynamicModule
from session_manager import SessionManager
from crisis_manager import CrisisManager
from documentation import DocumentationSystem
from models import Patient, Session, Assessment, TreatmentGoal
from config import Config, TherapyProtocols
from utils import log_action, format_datetime, sanitize_input

class AutomatedTherapySystem:
    """Fully automated therapy system using Gemini AI and existing therapy modules"""
    
    def __init__(self, gemini_api_key: str):
        # Initialize Gemini
        genai.configure(api_key=gemini_api_key)
        self.model = genai.GenerativeModel('gemini-2.0-flash-exp')
        
        # Initialize existing therapy system components
        self.db = TherapyDatabase()
        self.db.initialize_database()
        self.assessment_system = AssessmentSystem(self.db)
        self.goal_manager = GoalManager(self.db)
        self.homework_system = HomeworkSystem(self.db)
        self.therapy_modules = TherapyModuleIntegrator()
        self.session_manager = SessionManager(self.db)
        self.crisis_manager = CrisisManager(self.db)
        self.documentation = DocumentationSystem(self.db)
        
        print("Automated Therapy System initialized with all existing modules!")
    
    async def conduct_intake_session(self, patient_name: str, patient_dob: str = "1990-01-01") -> Dict[str, Any]:
        """Conduct automated intake session with assessment, goal setting, and treatment planning"""
        
        print(f"\nStarting automated intake for {patient_name}...")
        
        # Create patient using existing function
        patient_data = get_or_create_patient(
            self.db, 
            patient_name,
            date_of_birth=patient_dob,
            preferred_therapy_mode="CBT"
        )
        patient_id = patient_data['id']
        
        print(f"Patient created: ID {patient_id}")
        
        # Step 1: Initial conversation to understand patient's needs
        intake_conversation = await self._conduct_intake_conversation(patient_name)
        
        # Step 2: Automated assessment based on conversation
        assessment_results = await self._conduct_automated_assessments(patient_id, intake_conversation)
        
        # Step 3: AI determines appropriate therapy modality
        therapy_modality = await self._determine_therapy_modality(assessment_results, intake_conversation)
        
        # Step 4: Generate treatment goals
        goals = await self._generate_treatment_goals(patient_id, assessment_results, intake_conversation, therapy_modality)
        
        # Step 5: Create initial homework assignments
        homework = await self._generate_initial_homework(patient_id, therapy_modality, goals)
        
        # Step 6: Create treatment plan documentation
        treatment_plan = await self._create_treatment_plan(patient_id, therapy_modality, assessment_results, goals)
        
        return {
            'patient_id': patient_id,
            'patient_name': patient_name,
            'intake_conversation': intake_conversation,
            'assessment_results': assessment_results,
            'therapy_modality': therapy_modality,
            'goals': goals,
            'homework': homework,
            'treatment_plan': treatment_plan
        }
    
    async def _conduct_intake_conversation(self, patient_name: str) -> str:
        """Conduct AI-powered intake conversation"""
        
        intake_prompt = f"""You are conducting an initial therapy intake session with {patient_name}. 
        
        Your goal is to gather information about:
        1. Current mental health concerns and symptoms
        2. Life stressors and challenges
        3. Previous therapy or treatment experiences
        4. Goals and expectations for therapy
        5. Support systems and coping strategies
        6. Any crisis or safety concerns
        
        Conduct this as a natural, empathetic conversation. Ask open-ended questions and be supportive.
        Start with: "Hello {patient_name}, I'm here to help you today. Can you tell me what brings you to therapy?"
        
        After the conversation, provide a summary of key information gathered."""
        
        print("Starting intake conversation...")
        print("(In a real implementation, this would be interactive)")
        
        # Simulated conversation - in real use, this would be interactive
        conversation_summary = f"""
        Intake conversation with {patient_name}:
        
        Patient reports feeling overwhelmed with work stress and experiencing anxiety symptoms including:
        - Difficulty sleeping (3-4 hours per night)
        - Racing thoughts, especially about work performance
        - Physical symptoms: tight chest, rapid heartbeat
        - Avoiding social situations due to worry
        - Perfectionist tendencies causing procrastination
        
        Patient goals:
        - Better sleep and anxiety management
        - Improved work-life balance
        - Learning coping strategies for perfectionism
        - Building confidence in social situations
        
        No immediate safety concerns identified.
        Previous therapy experience: Brief counseling in college, found it helpful.
        Support system: Close relationship with partner, some family support.
        """
        
        return conversation_summary
    
    async def _conduct_automated_assessments(self, patient_id: int, conversation: str) -> Dict[str, Any]:
        """Use AI to determine which assessments to run and automatically complete them based on conversation"""
        
        # AI determines which assessments are needed
        assessment_prompt = f"""Based on this intake conversation, determine which assessments would be most appropriate:
        
        {conversation}
        
        Available assessments: PHQ9 (depression), GAD7 (anxiety), PCL5 (PTSD), ORS (outcomes), SRS (session rating)
        
        For each recommended assessment, provide:
        1. Assessment type
        2. Reason for selection
        3. Expected responses based on the conversation (simulate patient responses)
        
        Format as JSON with assessment_type, reason, and simulated_responses."""
        
        response = await self.model.generate_content_async(assessment_prompt)
        
        # Parse AI response and conduct assessments
        assessment_results = {}
        
        # Conduct PHQ9 based on conversation analysis
        if "anxiety" in conversation.lower() or "depression" in conversation.lower():
            phq9_result = await self._simulate_assessment_responses("PHQ9", conversation)
            assessment_id = save_assessment_result(
                self.db, patient_id, "PHQ9", 
                phq9_result['responses'], 
                phq9_result['total_score'],
                phq9_result['severity'], 
                phq9_result['interpretation']
            )
            assessment_results['PHQ9'] = phq9_result
        
        # Conduct GAD7 based on conversation analysis  
        if "anxiety" in conversation.lower() or "worry" in conversation.lower():
            gad7_result = await self._simulate_assessment_responses("GAD7", conversation)
            assessment_id = save_assessment_result(
                self.db, patient_id, "GAD7",
                gad7_result['responses'],
                gad7_result['total_score'], 
                gad7_result['severity'],
                gad7_result['interpretation']
            )
            assessment_results['GAD7'] = gad7_result
        
        return assessment_results
    
    async def _simulate_assessment_responses(self, assessment_type: str, conversation: str) -> Dict[str, Any]:
        """Use AI to simulate realistic assessment responses based on intake conversation"""
        
        # Get assessment questions from existing system
        assessment_data = self.assessment_system.get_assessment(assessment_type)
        
        simulation_prompt = f"""Based on this patient conversation, simulate realistic responses to the {assessment_type} assessment:
        
        Conversation: {conversation}
        
        Assessment: {assessment_data['name']}
        Instructions: {assessment_data['instructions']}
        
        For each question, select the most appropriate response option (0-{len(assessment_data['questions'][0]['options'])-1}) based on the patient's reported symptoms and experiences.
        
        Questions and options:
        """
        
        for i, q in enumerate(assessment_data['questions']):
            simulation_prompt += f"\nQuestion {i+1}: {q['text']}\nOptions: {q['options']}\n"
        
        simulation_prompt += "\nProvide responses as JSON: {'responses': {'q1': {'answer': 'option_text', 'score': number}, ...}, 'reasoning': 'explanation'}"
        
        response = await self.model.generate_content_async(simulation_prompt)
        
        # Parse AI response and calculate scores
        try:
            # Extract JSON from AI response
            json_match = re.search(r'\{.*\}', response.text, re.DOTALL)
            if json_match:
                ai_responses = json.loads(json_match.group())
                responses = ai_responses['responses']
            else:
                # Fallback: create moderate responses
                responses = {}
                for i in range(len(assessment_data['questions'])):
                    responses[f'q{i+1}'] = {'answer': assessment_data['questions'][i]['options'][1], 'score': 1}
        except:
            # Fallback responses
            responses = {}
            for i in range(len(assessment_data['questions'])):
                responses[f'q{i+1}'] = {'answer': assessment_data['questions'][i]['options'][1], 'score': 1}
        
        # Calculate total score
        total_score = sum(r['score'] for r in responses.values())
        
        # Get severity and interpretation using existing system
        severity = self.assessment_system.calculate_severity(assessment_type, total_score)
        interpretation = self.assessment_system.get_interpretation(assessment_type, total_score)
        
        return {
            'responses': responses,
            'total_score': total_score,
            'severity': severity,
            'interpretation': interpretation,
            'assessment_type': assessment_type
        }
    
    async def _determine_therapy_modality(self, assessment_results: Dict, conversation: str) -> str:
        """Use AI to determine the most appropriate therapy modality"""
        
        modality_prompt = f"""Based on the assessment results and intake conversation, recommend the most appropriate therapy modality:
        
        Assessment Results: {json.dumps(assessment_results, indent=2)}
        
        Conversation Summary: {conversation}
        
        Available modalities:
        - CBT: Best for depression, anxiety, negative thought patterns, behavioral changes
        - DBT: Best for emotional regulation, interpersonal issues, crisis behaviors, borderline traits
        - ACT: Best for values-based living, acceptance, psychological flexibility, chronic conditions
        - Psychodynamic: Best for insight-oriented work, relationship patterns, unconscious conflicts
        
        Respond with ONLY the modality name: CBT, DBT, ACT, or Psychodynamic"""
        
        try:
            response = await self.model.generate_content_async(modality_prompt)
            
            # Extract modality from response
            response_text = response.text.strip().upper()
            
            # Check for valid modalities
            valid_modalities = ['CBT', 'DBT', 'ACT', 'PSYCHODYNAMIC']
            for modality in valid_modalities:
                if modality in response_text:
                    return modality
            
        except Exception as e:
            print(f"AI modality determination error: {e}")
        
        # Fallback logic based on assessment scores
        if assessment_results:
            if 'GAD7' in assessment_results and assessment_results['GAD7']['total_score'] >= 10:
                return 'CBT'
            elif 'PHQ9' in assessment_results and assessment_results['PHQ9']['total_score'] >= 10:
                return 'CBT'
        
        return 'CBT'  # Default fallback
    
    async def _generate_treatment_goals(self, patient_id: int, assessment_results: Dict, 
                                       conversation: str, therapy_modality: str) -> List[Dict]:
        """Generate SMART treatment goals using AI"""
        
        goals_prompt = f"""Generate 3-5 SMART treatment goals based on:
        
        Assessment Results: {json.dumps(assessment_results, indent=2)}
        Conversation: {conversation}
        Therapy Modality: {therapy_modality}
        
        Goals should be:
        - Specific, Measurable, Achievable, Relevant, Time-bound
        - Categorized as: symptom, functional, or behavioral
        - Appropriate for {therapy_modality} approach
        
        For each goal provide:
        - goal_type: symptom/functional/behavioral  
        - description: Clear, specific goal statement
        - target_date: Date 3-6 months from now
        - measurement_criteria: How progress will be measured
        
        Format as JSON array."""
        
        response = await self.model.generate_content_async(goals_prompt)
        
        # Parse and create goals
        goals = []
        try:
            json_match = re.search(r'\[.*\]', response.text, re.DOTALL)
            if json_match:
                ai_goals = json.loads(json_match.group())
                
                for goal_data in ai_goals[:5]:  # Limit to 5 goals
                    goal_id = self.goal_manager.create_goal(
                        patient_id=patient_id,
                        goal_type=goal_data.get('goal_type', 'symptom'),
                        description=goal_data.get('description', 'Improve overall wellbeing'),
                        target_date=(datetime.now() + timedelta(days=120)).strftime('%Y-%m-%d'),
                        measurement_criteria=goal_data.get('measurement_criteria', 'Self-report improvement')
                    )
                    
                    goals.append({
                        'goal_id': goal_id,
                        'goal_type': goal_data.get('goal_type'),
                        'description': goal_data.get('description'),
                        'measurement_criteria': goal_data.get('measurement_criteria')
                    })
        except:
            # Fallback goal
            goal_id = self.goal_manager.create_goal(
                patient_id=patient_id,
                goal_type='symptom',
                description='Reduce anxiety symptoms and improve daily functioning',
                target_date=(datetime.now() + timedelta(days=90)).strftime('%Y-%m-%d'),
                measurement_criteria='Weekly anxiety rating scale, daily functioning checklist'
            )
            goals.append({
                'goal_id': goal_id,
                'goal_type': 'symptom',
                'description': 'Reduce anxiety symptoms and improve daily functioning'
            })
        
        return goals
    
    async def _generate_initial_homework(self, patient_id: int, therapy_modality: str, goals: List[Dict]) -> List[Dict]:
        """Generate initial homework assignments using therapy modules"""
        
        homework_assignments = []
        
        # Use existing therapy modules to generate appropriate homework
        if therapy_modality == 'CBT':
            cbt_module = CBTModule()
            
            # Create thought record homework
            homework_id = self.db.execute_update('''
                INSERT INTO homework_assignments 
                (patient_id, assignment_type, description, instructions, assigned_date, due_date)
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (
                patient_id,
                'thought_record',
                'Daily thought record for anxiety management',
                'Track anxious thoughts daily using the thought record worksheet. Note situation, thoughts, emotions, and evidence for/against the thoughts.',
                datetime.now().isoformat(),
                (datetime.now() + timedelta(days=7)).isoformat()
            ))
            
            homework_assignments.append({
                'homework_id': homework_id,
                'type': 'thought_record',
                'description': 'Daily thought record for anxiety management'
            })
            
        elif therapy_modality == 'DBT':
            # Create mindfulness homework
            homework_id = self.db.execute_update('''
                INSERT INTO homework_assignments 
                (patient_id, assignment_type, description, instructions, assigned_date, due_date)
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (
                patient_id,
                'mindfulness_practice',
                'Daily mindfulness exercises',
                'Practice 5-10 minutes of mindfulness daily using guided exercises. Focus on present moment awareness.',
                datetime.now().isoformat(),
                (datetime.now() + timedelta(days=7)).isoformat()
            ))
            
            homework_assignments.append({
                'homework_id': homework_id,
                'type': 'mindfulness_practice',
                'description': 'Daily mindfulness exercises'
            })
        
        return homework_assignments
    
    async def _create_treatment_plan(self, patient_id: int, therapy_modality: str, 
                                   assessment_results: Dict, goals: List[Dict]) -> Dict:
        """Create comprehensive treatment plan documentation"""
        
        # Convert data to JSON strings for database storage
        target_symptoms_json = json.dumps(list(assessment_results.keys()))
        treatment_goals_json = json.dumps([g.get('description', 'Goal') for g in goals])
        
        treatment_plan_id = self.db.execute_update('''
            INSERT INTO treatment_plans
            (patient_id, plan_name, primary_modality, target_symptoms, treatment_goals, 
             estimated_duration, session_frequency, created_date, status)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            patient_id,
            f"{therapy_modality} Treatment Plan",
            therapy_modality,
            target_symptoms_json,
            treatment_goals_json,
            12,  # 12 weeks
            'weekly',
            datetime.now().isoformat(),
            'active'
        ))
        
        return {
            'plan_id': treatment_plan_id,
            'modality': therapy_modality,
            'goals_count': len(goals),
            'estimated_duration': '12 weeks'
        }
    
    async def conduct_follow_up_session(self, patient_id: int) -> Dict[str, Any]:
        """Conduct automated follow-up session with progress tracking"""
        
        print(f"Conducting follow-up session for patient {patient_id}...")
        
        # Get patient information
        patient_data = self.db.execute_query("SELECT * FROM patients WHERE id = ?", (patient_id,))[0]
        
        # Check homework completion
        homework_status = await self._check_homework_progress(patient_id)
        
        # Update goal progress
        goal_updates = await self._update_goal_progress(patient_id)
        
        # Assess current symptoms
        current_assessment = await self._conduct_progress_assessment(patient_id)
        
        # Adjust treatment plan if needed
        treatment_adjustments = await self._adjust_treatment_plan(patient_id, homework_status, goal_updates, current_assessment)
        
        # Generate new homework
        new_homework = await self._generate_follow_up_homework(patient_id, treatment_adjustments)
        
        # Create session documentation
        session_id = create_session_record(
            self.db, patient_id, patient_data['preferred_therapy_mode'],
            duration=45,
            therapist_notes=f"Follow-up session completed. Progress assessed across goals and homework."
        )
        
        return {
            'session_id': session_id,
            'patient_id': patient_id,
            'homework_status': homework_status,
            'goal_updates': goal_updates,
            'current_assessment': current_assessment,
            'treatment_adjustments': treatment_adjustments,
            'new_homework': new_homework
        }
    
    async def _check_homework_progress(self, patient_id: int) -> Dict[str, Any]:
        """Check and update homework completion status"""
        
        # Get pending homework
        homework_list = self.db.execute_query(
            "SELECT * FROM homework_assignments WHERE patient_id = ? AND completed = FALSE",
            (patient_id,)
        )
        
        progress_summary = {
            'total_assignments': len(homework_list),
            'completed': 0,
            'overdue': 0,
            'current': 0
        }
        
        for hw in homework_list:
            due_date = datetime.fromisoformat(hw['due_date'])
            if due_date < datetime.now():
                progress_summary['overdue'] += 1
            else:
                progress_summary['current'] += 1
        
        return progress_summary
    
    async def _update_goal_progress(self, patient_id: int) -> List[Dict]:
        """AI-assisted goal progress updates"""
        
        # Use direct database query instead of goal_manager method to avoid column issues
        goals = self.db.execute_query(
            "SELECT * FROM treatment_goals WHERE patient_id = ? AND status = ? ORDER BY created_date DESC",
            (patient_id, 'active')
        )
        updates = []
        
        for goal in goals:
            # Simulate progress assessment (in real implementation, this would be based on patient input)
            estimated_progress = min(goal['current_progress'] + 15, 100)  # Assume some progress
            
            # Update progress directly in database
            self.db.execute_update('''
                UPDATE treatment_goals 
                SET current_progress = ?, last_updated = ?
                WHERE id = ?
            ''', (
                estimated_progress,
                datetime.now().isoformat(),
                goal['id']
            ))
            
            updates.append({
                'goal_id': goal['id'],
                'description': goal['goal_description'],
                'old_progress': goal['current_progress'],
                'new_progress': estimated_progress,
                'updated': True
            })
        
        return updates
    
    async def _conduct_progress_assessment(self, patient_id: int) -> Dict[str, Any]:
        """Conduct brief progress assessment"""
        
        # Get latest assessment scores
        latest_assessments = self.db.execute_query(
            "SELECT * FROM assessments WHERE patient_id = ? ORDER BY assessment_date DESC LIMIT 5",
            (patient_id,)
        )
        
        return {
            'latest_scores': {a['assessment_type']: a['total_score'] for a in latest_assessments},
            'trend': 'stable',  # Would be calculated based on score history
            'recommendations': ['Continue current interventions', 'Monitor progress weekly']
        }
    
    async def _adjust_treatment_plan(self, patient_id: int, homework_status: Dict, 
                                   goal_updates: List[Dict], assessment: Dict) -> Dict[str, Any]:
        """AI-assisted treatment plan adjustments"""
        
        adjustments = {
            'homework_adjustments': [],
            'goal_adjustments': [],
            'modality_changes': [],
            'session_frequency': 'weekly'
        }
        
        # Adjust based on homework compliance
        if homework_status['overdue'] > 0:
            adjustments['homework_adjustments'].append('Reduce homework complexity')
            adjustments['homework_adjustments'].append('Focus on engagement strategies')
        
        # Adjust based on goal progress
        completed_goals = [g for g in goal_updates if g['new_progress'] >= 100]
        if completed_goals:
            adjustments['goal_adjustments'].append(f'Celebrate {len(completed_goals)} completed goals')
            adjustments['goal_adjustments'].append('Consider new goal areas')
        
        return adjustments
    
    async def _generate_follow_up_homework(self, patient_id: int, adjustments: Dict) -> List[Dict]:
        """Generate follow-up homework based on progress"""
        
        # Get patient's therapy modality
        patient = self.db.execute_query("SELECT * FROM patients WHERE id = ?", (patient_id,))[0]
        modality = patient['preferred_therapy_mode']
        
        homework_assignments = []
        
        if modality == 'CBT':
            homework_id = self.db.execute_update('''
                INSERT INTO homework_assignments 
                (patient_id, assignment_type, description, instructions, assigned_date, due_date)
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (
                patient_id,
                'behavioral_activation',
                'Activity scheduling for mood improvement',
                'Schedule and complete 2 pleasant activities and 2 mastery activities this week.',
                datetime.now().isoformat(),
                (datetime.now() + timedelta(days=7)).isoformat()
            ))
            
            homework_assignments.append({
                'homework_id': homework_id,
                'type': 'behavioral_activation',
                'description': 'Activity scheduling for mood improvement'
            })
        
        return homework_assignments

# Main execution functions
async def run_intake_demo(api_key: str, patient_name: str = "Alex Johnson"):
    """Run a complete intake session demonstration"""
    
    system = AutomatedTherapySystem(api_key)
    
    print("=" * 60)
    print("AUTOMATED THERAPY INTAKE DEMONSTRATION")
    print("=" * 60)
    
    # Conduct intake session
    intake_results = await system.conduct_intake_session(patient_name)
    
    print("\nIntake Session Complete!")
    print(f"Patient: {intake_results['patient_name']} (ID: {intake_results['patient_id']})")
    print(f"Recommended Therapy: {intake_results['therapy_modality']}")
    print(f"Goals Created: {len(intake_results['goals'])}")
    print(f"Homework Assigned: {len(intake_results['homework'])}")
    
    # Display assessment results
    print("\nAssessment Results:")
    for assessment_type, result in intake_results['assessment_results'].items():
        print(f"  {assessment_type}: Score {result['total_score']} ({result['severity']})")
    
    # Display goals
    print("\nTreatment Goals:")
    for i, goal in enumerate(intake_results['goals'], 1):
        print(f"  {i}. [{goal['goal_type'].title()}] {goal['description']}")
    
    return system, intake_results['patient_id']

async def run_follow_up_demo(system: AutomatedTherapySystem, patient_id: int):
    """Run a follow-up session demonstration"""
    
    print("\n" + "=" * 60)
    print("AUTOMATED FOLLOW-UP SESSION DEMONSTRATION")
    print("=" * 60)
    
    # Conduct follow-up session
    followup_results = await system.conduct_follow_up_session(patient_id)
    
    print("\nFollow-up Session Complete!")
    print(f"Session ID: {followup_results['session_id']}")
    print(f"Homework Status: {followup_results['homework_status']}")
    print(f"Goals Updated: {len(followup_results['goal_updates'])}")
    print(f"New Homework: {len(followup_results['new_homework'])}")
    
    # Display goal progress
    print("\nGoal Progress Updates:")
    for update in followup_results['goal_updates']:
        print(f"  Goal: {update['description'][:50]}...")
        print(f"  Progress: {update['old_progress']}% → {update['new_progress']}%")
    
    return followup_results

# Usage Instructions
print("""
AUTOMATED AI THERAPY SYSTEM - JUPYTER NOTEBOOK

This notebook provides a fully automated therapy system that:
1. Conducts intake sessions with AI conversation
2. Automatically determines appropriate assessments
3. Selects optimal therapy modality (CBT, DBT, ACT, Psychodynamic)
4. Generates SMART treatment goals
5. Creates personalized homework assignments
6. Tracks progress and adjusts treatment plans
7. Conducts follow-up sessions automatically

TO USE:
1. Set your Gemini API key in the environment or pass it directly
2. Run the intake demonstration
3. Run follow-up sessions to see progress tracking

EXAMPLE USAGE:
```python
# Set your API key
api_key = "your-gemini-api-key-here"

# Run intake session
system, patient_id = await run_intake_demo(api_key, "John Smith")

# Run follow-up session
followup_results = await run_follow_up_demo(system, patient_id)
```

The system integrates with all your existing therapy modules and database.
""")

# Example execution (uncomment to run)
# import asyncio
# 
# async def main():
#     # Replace with your actual Gemini API key
#     api_key = "your-gemini-api-key-here"  
#     
#     # Run intake session
#     system, patient_id = await run_intake_demo(api_key, "Sarah Wilson")
#     
#     # Simulate some time passing
#     print("\n⏳ Simulating 1 week later...")
#     
#     # Run follow-up session
#     followup_results = await run_follow_up_demo(system, patient_id)
#     
#     return system
# 
# # Run the demonstration
# # system = await main()


AUTOMATED AI THERAPY SYSTEM - JUPYTER NOTEBOOK

This notebook provides a fully automated therapy system that:
1. Conducts intake sessions with AI conversation
2. Automatically determines appropriate assessments
3. Selects optimal therapy modality (CBT, DBT, ACT, Psychodynamic)
4. Generates SMART treatment goals
5. Creates personalized homework assignments
6. Tracks progress and adjusts treatment plans
7. Conducts follow-up sessions automatically

TO USE:
1. Set your Gemini API key in the environment or pass it directly
2. Run the intake demonstration
3. Run follow-up sessions to see progress tracking

EXAMPLE USAGE:
```python
# Set your API key
api_key = "your-gemini-api-key-here"

# Run intake session
system, patient_id = await run_intake_demo(api_key, "John Smith")

# Run follow-up session
followup_results = await run_follow_up_demo(system, patient_id)
```

The system integrates with all your existing therapy modules and database.



In [ ]:

import asyncio

async def main():
     # Replace with your actual Gemini API key
     api_key = "api-key-here"  
     
     # Run intake session
     system, patient_id = await run_intake_demo(api_key, "Sarah Wilson")
     
     # Simulate some time passing
     print("\n⏳ Simulating 1 week later...")
     
     # Run follow-up session
     followup_results = await run_follow_up_demo(system, patient_id)
     
     return system
 
 # Run the demonstration
system = await main()

Automated Therapy System initialized with all existing modules!
AUTOMATED THERAPY INTAKE DEMONSTRATION

Starting automated intake for Sarah Wilson...
Patient created: ID 2
Starting intake conversation...
(In a real implementation, this would be interactive)

Intake Session Complete!
Patient: Sarah Wilson (ID: 2)
Recommended Therapy: CBT
Goals Created: 4
Homework Assigned: 1

Assessment Results:
  PHQ9: Score 8 (Mild)
  GAD7: Score 14 (Moderate)

Treatment Goals:
  1. [Symptom] Reduce anxiety symptoms as measured by the GAD-7 from a score of 14 (moderate) to 7 or below (mild) to improve overall well-being.
  2. [Symptom] Improve sleep duration by increasing average nightly sleep from 3-4 hours to at least 6 hours to enhance daily functioning and reduce anxiety.
  3. [Behavioral] Engage in at least one social activity per week (e.g., meeting with a friend, attending a club meeting) to decrease social avoidance related to anxiety.
  4. [Functional] Reduce procrastination related to perfec

In [8]:
# Interactive AI Therapy System - Jupyter Notebook
# Real-time conversational therapy system with automated assessment and treatment planning

import os
import sys
import json
import asyncio
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional
import google.generativeai as genai
import re

# Add the therapy system to path
sys.path.append('.')

# Import all existing therapy system modules
from database import TherapyDatabase, get_or_create_patient, create_session_record, save_assessment_result
from assessment_system import AssessmentSystem
from goal_manager import GoalManager
from homework_system import HomeworkSystem
from therapy_modules import TherapyModuleIntegrator, CBTModule, DBTModule, ACTModule, PsychodynamicModule
from session_manager import SessionManager
from crisis_manager import CrisisManager
from documentation import DocumentationSystem
from models import Patient, Session, Assessment, TreatmentGoal
from config import Config, TherapyProtocols
from utils import log_action, format_datetime, sanitize_input

class InteractiveTherapySystem:
    """Interactive AI therapy system with real-time conversation and automated treatment planning"""
    
    def __init__(self, gemini_api_key: str):
        # Initialize Gemini
        genai.configure(api_key=gemini_api_key)
        self.model = genai.GenerativeModel('gemini-2.0-flash-exp')
        
        # Initialize therapy system components
        self.db = TherapyDatabase()
        self.db.initialize_database()
        self.assessment_system = AssessmentSystem(self.db)
        self.goal_manager = GoalManager(self.db)
        self.homework_system = HomeworkSystem(self.db)
        self.therapy_modules = TherapyModuleIntegrator()
        self.session_manager = SessionManager(self.db)
        self.crisis_manager = CrisisManager(self.db)
        self.documentation = DocumentationSystem(self.db)
        
        # Session state
        self.current_patient = None
        self.current_session_phase = "intake"
        self.conversation_history = []
        self.detected_symptoms = []
        self.assessment_responses = {}
        
        print("Interactive Therapy System ready!")
    
    async def start_interactive_session(self, patient_name: str) -> None:
        """Start an interactive therapy session"""
        
        # Create or get patient
        patient_data = get_or_create_patient(
            self.db, 
            patient_name,
            preferred_therapy_mode="CBT"  # Will be determined during session
        )
        
        self.current_patient = patient_data
        print(f"\n--- Starting Interactive Session with {patient_name} (ID: {patient_data['id']}) ---")
        print("Type 'quit' to end session, 'help' for commands, or just chat naturally!\n")
        
        # Start with AI greeting
        greeting = await self._get_ai_response("greeting", "")
        print(f"AI Therapist: {greeting}")
        
        # Interactive conversation loop
        while True:
            try:
                user_input = input(f"\n{patient_name}: ").strip()
                
                if user_input.lower() in ['quit', 'exit', 'end']:
                    await self._end_interactive_session()
                    break
                elif user_input.lower() == 'help':
                    self._show_help()
                    continue
                elif user_input.lower() == 'status':
                    await self._show_session_status()
                    continue
                elif user_input.lower().startswith('mood '):
                    await self._record_mood(user_input)
                    continue
                elif not user_input:
                    print("Please say something or type 'help' for commands.")
                    continue
                
                # Process user input and get AI response
                response = await self._process_user_message(user_input)
                print(f"\nAI Therapist: {response}")
                
                # Check if we should transition phases or take automated actions
                await self._check_phase_transitions()
                
            except KeyboardInterrupt:
                print("\n\nSession interrupted. Saving progress...")
                await self._end_interactive_session()
                break
            except Exception as e:
                print(f"\nError: {e}")
                print("Continuing session...")
    
    async def _get_ai_response(self, context: str, user_input: str) -> str:
        """Get contextual AI response based on session phase and conversation history"""
        
        # Build conversation context
        recent_history = "\n".join([
            f"Patient: {msg['user']}\nTherapist: {msg['ai']}" 
            for msg in self.conversation_history[-3:]  # Last 3 exchanges
        ])
        
        # Determine current phase and appropriate prompt
        if context == "greeting":
            prompt = self._get_greeting_prompt()
        elif self.current_session_phase == "intake":
            prompt = self._get_intake_prompt(user_input, recent_history)
        elif self.current_session_phase == "assessment":
            prompt = self._get_assessment_prompt(user_input, recent_history)
        elif self.current_session_phase == "therapy":
            prompt = self._get_therapy_prompt(user_input, recent_history)
        elif self.current_session_phase == "goal_setting":
            prompt = self._get_goal_setting_prompt(user_input, recent_history)
        else:
            prompt = self._get_general_prompt(user_input, recent_history)
        
        # Get AI response
        try:
            response = await self.model.generate_content_async(prompt)
            ai_response = response.text.strip()
            
            # Store conversation
            self.conversation_history.append({
                'user': user_input,
                'ai': ai_response,
                'timestamp': datetime.now().isoformat(),
                'phase': self.current_session_phase
            })
            
            # Extract insights from response
            await self._analyze_conversation_for_insights(user_input, ai_response)
            
            return ai_response
            
        except Exception as e:
            return f"I'm having trouble processing that right now. Can you tell me more about what's on your mind? (Error: {e})"
    
    def _get_greeting_prompt(self) -> str:
        """Get initial greeting prompt"""
        return f"""You are a warm, professional AI therapist starting a session with {self.current_patient['name']}. 

Greet them naturally and begin exploring what brings them to therapy today. Be empathetic and use open-ended questions. Keep it conversational and welcoming.

Start with something like welcoming them and asking what's been on their mind lately."""
    
    def _get_intake_prompt(self, user_input: str, history: str) -> str:
        """Get intake phase prompt"""
        return f"""You are conducting an intake session with {self.current_patient['name']}. Your goal is to understand:
- Current mental health concerns and symptoms
- Life stressors and challenges  
- Goals for therapy
- Support systems
- Any safety concerns

Conversation so far:
{history}

Patient just said: "{user_input}"

Respond with empathy and ask thoughtful follow-up questions. Be a skilled therapist who listens carefully and guides the conversation naturally. If they mention symptoms of depression, anxiety, trauma, or other concerns, explore gently."""
    
    def _get_assessment_prompt(self, user_input: str, history: str) -> str:
        """Get assessment phase prompt"""
        detected_symptoms_text = ", ".join(self.detected_symptoms) if self.detected_symptoms else "general mental health"
        
        return f"""You are conducting a therapeutic assessment. Based on the conversation, you've identified potential concerns with: {detected_symptoms_text}

Conversation history:
{history}

Patient just said: "{user_input}"

Continue exploring their symptoms and experiences. Ask specific questions that would help you understand severity, frequency, and impact on their daily life. Be thorough but gentle."""
    
    def _get_therapy_prompt(self, user_input: str, history: str) -> str:
        """Get therapy phase prompt"""
        modality = self.current_patient.get('preferred_therapy_mode', 'CBT')
        
        return f"""You are conducting {modality} therapy. Use evidence-based {modality} techniques and approaches.

For CBT: Focus on thoughts, feelings, behaviors connection. Use cognitive restructuring, behavioral activation.
For DBT: Focus on mindfulness, distress tolerance, emotion regulation, interpersonal effectiveness.
For ACT: Focus on psychological flexibility, values, acceptance, mindfulness.
For Psychodynamic: Focus on insight, patterns, relationships, unconscious processes.

Conversation history:
{history}

Patient just said: "{user_input}"

Respond as a skilled {modality} therapist. Be therapeutic, insightful, and helpful."""
    
    def _get_goal_setting_prompt(self, user_input: str, history: str) -> str:
        """Get goal setting phase prompt"""
        return f"""You are helping {self.current_patient['name']} set therapeutic goals. Based on your conversation, help them identify:
- Specific, measurable goals
- Symptom reduction goals  
- Functional improvement goals
- Behavioral change goals

Conversation history:
{history}

Patient just said: "{user_input}"

Guide them to articulate clear, achievable goals. Ask about what they want to be different and how they'll know when they've made progress."""
    
    def _get_general_prompt(self, user_input: str, history: str) -> str:
        """Get general therapy prompt"""
        return f"""You are an AI therapist in session with {self.current_patient['name']}. Be supportive, insightful, and therapeutic.

Conversation history:
{history}

Patient just said: "{user_input}"

Respond thoughtfully as a professional therapist."""
    
    async def _process_user_message(self, user_input: str) -> str:
        """Process user message and return AI response"""
        
        # Check for crisis indicators
        await self._check_for_crisis(user_input)
        
        # Extract symptoms and insights
        await self._extract_symptoms_and_insights(user_input)
        
        # Get appropriate AI response based on current phase
        if self.current_session_phase == "intake":
            response = await self._get_ai_response("intake", user_input)
        elif self.current_session_phase == "assessment":
            response = await self._get_ai_response("assessment", user_input)
        elif self.current_session_phase == "therapy":
            response = await self._get_ai_response("therapy", user_input)
        elif self.current_session_phase == "goal_setting":
            response = await self._get_ai_response("goal_setting", user_input)
        else:
            response = await self._get_ai_response("general", user_input)
        
        return response
    
    async def _analyze_conversation_for_insights(self, user_input: str, ai_response: str) -> None:
        """Analyze conversation for therapeutic insights and automated actions"""
        
        # Extract symptoms mentioned
        symptom_keywords = {
            'anxiety': ['anxious', 'worried', 'panic', 'nervous', 'fear', 'racing thoughts'],
            'depression': ['depressed', 'sad', 'hopeless', 'empty', 'worthless', 'tired'],
            'sleep': ['sleep', 'insomnia', 'tired', 'exhausted', 'wake up'],
            'social': ['social', 'friends', 'lonely', 'isolated', 'avoid people'],
            'work': ['work', 'job', 'boss', 'career', 'stressed at work']
        }
        
        user_lower = user_input.lower()
        for symptom, keywords in symptom_keywords.items():
            if any(keyword in user_lower for keyword in keywords):
                if symptom not in self.detected_symptoms:
                    self.detected_symptoms.append(symptom)
    
    async def _check_for_crisis(self, user_input: str) -> None:
        """Check for crisis indicators"""
        crisis_keywords = ['suicide', 'kill myself', 'end it all', 'hurt myself', 'die', 'death']
        
        if any(keyword in user_input.lower() for keyword in crisis_keywords):
            print("\n🚨 CRISIS ALERT: Safety concerns detected. Activating crisis protocols.")
            print("If you're in immediate danger, please call 988 (Suicide & Crisis Lifeline) or 911.")
            
            # Log crisis
            log_action(f"Crisis keywords detected in interactive session", "crisis", 
                      patient_id=self.current_patient['id'])
    
    async def _extract_symptoms_and_insights(self, user_input: str) -> None:
        """Extract symptoms and insights from user input"""
        
        # Simple keyword-based extraction (could be enhanced with AI)
        insights = {
            'mood_indicators': [],
            'behavioral_patterns': [],
            'cognitive_patterns': [],
            'social_patterns': []
        }
        
        user_lower = user_input.lower()
        
        # Mood indicators
        if any(word in user_lower for word in ['sad', 'down', 'depressed', 'low']):
            insights['mood_indicators'].append('low_mood')
        if any(word in user_lower for word in ['anxious', 'worried', 'nervous', 'panic']):
            insights['mood_indicators'].append('anxiety')
        
        # Behavioral patterns
        if any(word in user_lower for word in ['avoid', 'staying home', 'cancelled', 'didn\'t go']):
            insights['behavioral_patterns'].append('avoidance')
        if any(word in user_lower for word in ['can\'t sleep', 'insomnia', 'staying up']):
            insights['behavioral_patterns'].append('sleep_disturbance')
        
        # Store insights for later use
        if not hasattr(self, 'session_insights'):
            self.session_insights = []
        
        for category, items in insights.items():
            for item in items:
                if item not in [i['item'] for i in self.session_insights]:
                    self.session_insights.append({
                        'category': category,
                        'item': item,
                        'timestamp': datetime.now().isoformat()
                    })
    
    async def _check_phase_transitions(self) -> None:
        """Check if we should transition to a different session phase"""
        
        # Simple heuristics for phase transitions
        conversation_count = len(self.conversation_history)
        
        if self.current_session_phase == "intake" and conversation_count >= 8:
            print("\n[System: Transitioning to assessment phase based on conversation depth]")
            self.current_session_phase = "assessment"
            await self._auto_conduct_assessments()
            
        elif self.current_session_phase == "assessment" and conversation_count >= 12:
            print("\n[System: Transitioning to therapy phase]")
            self.current_session_phase = "therapy"
            therapy_modality = await self._determine_therapy_modality()
            print(f"[System: Recommended therapy approach: {therapy_modality}]")
            
        elif self.current_session_phase == "therapy" and conversation_count >= 18:
            print("\n[System: Transitioning to goal setting phase]")
            self.current_session_phase = "goal_setting"
            
        elif self.current_session_phase == "goal_setting" and conversation_count >= 22:
            print("\n[System: Session approaching completion - generating treatment plan]")
            await self._auto_generate_treatment_plan()
    
    async def _auto_conduct_assessments(self) -> None:
        """Automatically conduct assessments based on detected symptoms"""
        
        print("\n[System: Conducting automated assessments based on conversation...]")
        
        # Determine which assessments to run
        assessments_to_run = []
        if 'anxiety' in self.detected_symptoms:
            assessments_to_run.append('GAD7')
        if 'depression' in self.detected_symptoms:
            assessments_to_run.append('PHQ9')
        
        if not assessments_to_run:
            assessments_to_run = ['PHQ9']  # Default
        
        # Simulate assessment responses based on conversation
        for assessment_type in assessments_to_run:
            result = await self._simulate_assessment_from_conversation(assessment_type)
            
            # Save assessment
            assessment_id = save_assessment_result(
                self.db, 
                self.current_patient['id'], 
                assessment_type,
                result['responses'],
                result['total_score'],
                result['severity'],
                result['interpretation']
            )
            
            print(f"[System: {assessment_type} completed - Score: {result['total_score']} ({result['severity']})]")
    
    async def _simulate_assessment_from_conversation(self, assessment_type: str) -> Dict[str, Any]:
        """Simulate assessment responses based on conversation content"""
        
        # Get assessment questions
        assessment_data = self.assessment_system.get_assessment(assessment_type)
        
        # AI prompt to simulate responses
        conversation_text = "\n".join([
            f"Patient: {msg['user']}" for msg in self.conversation_history
        ])
        
        simulation_prompt = f"""Based on this therapy conversation, simulate realistic {assessment_type} assessment responses:

Conversation:
{conversation_text}

Assessment: {assessment_data['name']}
Instructions: {assessment_data['instructions']}

For each question, select the most appropriate response (0-3 or 0-4) based on the patient's described symptoms and experiences:

"""
        
        for i, q in enumerate(assessment_data['questions']):
            simulation_prompt += f"\nQ{i+1}: {q['text']}\nOptions: {q['options']}\n"
        
        simulation_prompt += "\nRespond with just the numbers (e.g., '2 1 3 1 0 2 1 2 3' for PHQ9):"
        
        try:
            response = await self.model.generate_content_async(simulation_prompt)
            
            # Parse numbers from response
            numbers = re.findall(r'\d', response.text)
            
            # Create responses dict
            responses = {}
            total_score = 0
            
            for i, score_str in enumerate(numbers[:len(assessment_data['questions'])]):
                score = int(score_str)
                if i < len(assessment_data['questions']):
                    q = assessment_data['questions'][i]
                    if score < len(q['options']):
                        responses[f'q{i+1}'] = {
                            'answer': q['options'][score],
                            'score': q['scores'][score] if score < len(q['scores']) else score
                        }
                        total_score += responses[f'q{i+1}']['score']
            
            # Get interpretation
            severity = self.assessment_system.calculate_severity(assessment_type, total_score)
            interpretation = self.assessment_system.get_interpretation(assessment_type, total_score)
            
            return {
                'responses': responses,
                'total_score': total_score,
                'severity': severity,
                'interpretation': interpretation,
                'assessment_type': assessment_type
            }
            
        except Exception as e:
            # Fallback: moderate responses
            responses = {}
            total_score = 0
            for i in range(len(assessment_data['questions'])):
                responses[f'q{i+1}'] = {'answer': assessment_data['questions'][i]['options'][1], 'score': 1}
                total_score += 1
            
            severity = self.assessment_system.calculate_severity(assessment_type, total_score)
            interpretation = self.assessment_system.get_interpretation(assessment_type, total_score)
            
            return {
                'responses': responses,
                'total_score': total_score,
                'severity': severity,
                'interpretation': interpretation,
                'assessment_type': assessment_type
            }
    
    async def _determine_therapy_modality(self) -> str:
        """Determine appropriate therapy modality based on conversation"""
        
        conversation_summary = "\n".join([
            f"Patient: {msg['user']}\nTherapist: {msg['ai']}" 
            for msg in self.conversation_history
        ])
        
        modality_prompt = f"""Based on this therapy conversation, recommend the most appropriate therapy modality:

Conversation:
{conversation_summary}

Detected symptoms: {', '.join(self.detected_symptoms)}

Available modalities:
- CBT: Best for depression, anxiety, negative thought patterns
- DBT: Best for emotional regulation, interpersonal issues, crisis behaviors  
- ACT: Best for values-based living, acceptance, psychological flexibility
- Psychodynamic: Best for insight-oriented work, relationship patterns

Respond with just the modality name: CBT, DBT, ACT, or Psychodynamic"""
        
        try:
            response = await self.model.generate_content_async(modality_prompt)
            modality = response.text.strip().upper()
            
            if modality in ['CBT', 'DBT', 'ACT', 'PSYCHODYNAMIC']:
                # Update patient record
                self.db.execute_update(
                    "UPDATE patients SET preferred_therapy_mode = ? WHERE id = ?",
                    (modality, self.current_patient['id'])
                )
                return modality
        except:
            pass
        
        return 'CBT'  # Fallback
    
    async def _auto_generate_treatment_plan(self) -> None:
        """Automatically generate treatment plan and goals"""
        
        print("\n[System: Generating personalized treatment plan...]")
        
        conversation_summary = "\n".join([
            msg['user'] for msg in self.conversation_history
        ])
        
        # Generate goals using AI
        goals_prompt = f"""Based on this therapy conversation, create 3-4 SMART treatment goals:

Patient conversation: {conversation_summary}
Detected symptoms: {', '.join(self.detected_symptoms)}
Therapy modality: {self.current_patient.get('preferred_therapy_mode', 'CBT')}

Create goals that are:
- Specific and measurable
- Address the patient's main concerns
- Appropriate for their therapy modality
- Achievable within 3-6 months

Format as:
1. [Type] Goal description with measurement
2. [Type] Goal description with measurement
etc.

Types: Symptom, Behavioral, or Functional"""
        
        try:
            response = await self.model.generate_content_async(goals_prompt)
            goals_text = response.text.strip()
            
            # Parse and create goals
            goal_lines = [line.strip() for line in goals_text.split('\n') if line.strip() and line[0].isdigit()]
            
            created_goals = []
            for goal_line in goal_lines[:4]:  # Max 4 goals
                # Extract goal type and description
                if '[' in goal_line and ']' in goal_line:
                    goal_type = goal_line.split('[')[1].split(']')[0].lower()
                    description = goal_line.split(']')[1].strip()
                else:
                    goal_type = 'symptom'
                    description = goal_line[2:].strip()  # Remove "1. " prefix
                
                # Create goal in database
                try:
                    goal_id = self.db.execute_update('''
                        INSERT INTO treatment_goals 
                        (patient_id, goal_type, goal_description, target_date, 
                         measurement_criteria, current_progress, status, created_date, last_updated)
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        self.current_patient['id'],
                        goal_type if goal_type in ['symptom', 'behavioral', 'functional'] else 'symptom',
                        description,
                        (datetime.now() + timedelta(days=120)).strftime('%Y-%m-%d'),
                        'Weekly self-report and therapist assessment',
                        0,
                        'active',
                        datetime.now().isoformat(),
                        datetime.now().isoformat()
                    ))
                    
                    created_goals.append(f"{goal_type.title()}: {description}")
                    
                except Exception as e:
                    print(f"[System: Error creating goal - {e}]")
            
            print(f"[System: Created {len(created_goals)} treatment goals:]")
            for i, goal in enumerate(created_goals, 1):
                print(f"  {i}. {goal}")
            
            # Generate homework assignment
            await self._auto_assign_homework()
            
        except Exception as e:
            print(f"[System: Error generating treatment plan - {e}]")
    
    async def _auto_assign_homework(self) -> None:
        """Automatically assign homework based on therapy modality"""
        
        modality = self.current_patient.get('preferred_therapy_mode', 'CBT')
        
        homework_assignments = {
            'CBT': {
                'type': 'thought_record',
                'description': 'Daily thought record for cognitive restructuring',
                'instructions': 'Track negative thoughts daily using the thought record worksheet. Note situation, automatic thoughts, emotions, and develop balanced alternative thoughts.'
            },
            'DBT': {
                'type': 'mindfulness_practice',
                'description': 'Daily mindfulness and emotion regulation practice',
                'instructions': 'Practice mindfulness exercises for 10 minutes daily and use distress tolerance skills when needed.'
            },
            'ACT': {
                'type': 'values_exploration',
                'description': 'Values-based activity scheduling',
                'instructions': 'Identify your core values and schedule 2-3 activities this week that align with those values.'
            },
            'PSYCHODYNAMIC': {
                'type': 'reflection_journal',
                'description': 'Daily reflection journal for pattern recognition',
                'instructions': 'Keep a daily journal noting emotional reactions, relationship patterns, and insights about recurring themes.'
            }
        }
        
        hw = homework_assignments.get(modality, homework_assignments['CBT'])
        
        try:
            homework_id = self.db.execute_update('''
                INSERT INTO homework_assignments 
                (patient_id, assignment_type, description, instructions, assigned_date, due_date)
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (
                self.current_patient['id'],
                hw['type'],
                hw['description'],
                hw['instructions'],
                datetime.now().isoformat(),
                (datetime.now() + timedelta(days=7)).isoformat()
            ))
            
            print(f"[System: Assigned {modality} homework - {hw['description']}]")
            
        except Exception as e:
            print(f"[System: Error assigning homework - {e}]")
    
    def _show_help(self) -> None:
        """Show available commands"""
        print("""
Available Commands:
- 'quit' or 'exit' - End the session
- 'help' - Show this help message  
- 'status' - Show current session status
- 'mood [1-10]' - Record current mood rating
- Just chat naturally - The AI will respond therapeutically
        """)
    
    async def _show_session_status(self) -> None:
        """Show current session status"""
        print(f"""
Session Status:
- Patient: {self.current_patient['name']} (ID: {self.current_patient['id']})
- Current Phase: {self.current_session_phase}
- Conversation Exchanges: {len(self.conversation_history)}
- Detected Symptoms: {', '.join(self.detected_symptoms) if self.detected_symptoms else 'None detected yet'}
- Session Duration: {len(self.conversation_history) * 2} minutes (estimated)
        """)
    
    async def _record_mood(self, user_input: str) -> None:
        """Record mood rating from user input"""
        try:
            mood_rating = int(user_input.split()[1])
            if 1 <= mood_rating <= 10:
                log_action(f"Mood rating recorded: {mood_rating}", "interactive_session", 
                          patient_id=self.current_patient['id'])
                print(f"[System: Mood rating {mood_rating}/10 recorded]")
            else:
                print("[System: Mood rating must be between 1 and 10]")
        except:
            print("[System: Invalid mood format. Use: mood [1-10]]")
    
    async def _end_interactive_session(self) -> None:
        """End the interactive session and save progress"""
        
        print(f"\n--- Ending Session with {self.current_patient['name']} ---")
        
        # Create session record
        session_id = create_session_record(
            self.db, 
            self.current_patient['id'], 
            self.current_patient.get('preferred_therapy_mode', 'CBT'),
            duration=len(self.conversation_history) * 2,  # Estimate 2 min per exchange
            therapist_notes=f"Interactive AI session completed. {len(self.conversation_history)} exchanges. Phase: {self.current_session_phase}"
        )
        
        print(f"Session saved (ID: {session_id})")
        print(f"Total conversation exchanges: {len(self.conversation_history)}")
        print(f"Detected symptoms: {', '.join(self.detected_symptoms) if self.detected_symptoms else 'None'}")
        print(f"Final phase: {self.current_session_phase}")
        
        # Show summary
        if hasattr(self, 'session_insights') and self.session_insights:
            print(f"Session insights gathered: {len(self.session_insights)}")
        
        print("\nSession completed. Thank you!")

# Interactive session launcher
async def start_interactive_therapy(api_key: str, patient_name: str = None):
    """Start an interactive therapy session"""
    
    system = InteractiveTherapySystem(api_key)
    
    if not patient_name:
        patient_name = input("Enter patient name: ").strip()
        if not patient_name:
            patient_name = "Anonymous Patient"
    
    await system.start_interactive_session(patient_name)
    return system

# Usage instructions
print("""
INTERACTIVE AI THERAPY SYSTEM

This system provides real-time conversational therapy with:
- Natural conversation with AI therapist
- Automatic symptom detection and assessment
- Dynamic therapy modality selection
- Real-time goal setting and treatment planning
- Automated homework assignment
- Crisis detection and safety protocols

TO START A SESSION:
```python
# Set your Gemini API key
api_key = "your-gemini-api-key-here"

# Start interactive session
system = await start_interactive_therapy(api_key, "Patient Name")
```

During the session:
- Chat naturally with the AI therapist
- The system automatically detects symptoms and adjusts approach
- It will conduct assessments, set goals, and assign homework
- Type 'help' for commands, 'status' for session info, 'quit' to end

The system integrates with your existing therapy database and modules.
""")

# Example usage (uncomment to run)
# import asyncio
# 
# async def demo_session():
#     api_key = "your-gemini-api-key-here"  # Replace with actual key
#     system = await start_interactive_therapy(api_key, "Interactive Demo User")
#     return system
# 
# # Run the interactive session
# # demo_system = await demo_session()


INTERACTIVE AI THERAPY SYSTEM

This system provides real-time conversational therapy with:
- Natural conversation with AI therapist
- Automatic symptom detection and assessment
- Dynamic therapy modality selection
- Real-time goal setting and treatment planning
- Automated homework assignment
- Crisis detection and safety protocols

TO START A SESSION:
```python
# Set your Gemini API key
api_key = "your-gemini-api-key-here"

# Start interactive session
system = await start_interactive_therapy(api_key, "Patient Name")
```

During the session:
- Chat naturally with the AI therapist
- The system automatically detects symptoms and adjusts approach
- It will conduct assessments, set goals, and assign homework
- Type 'help' for commands, 'status' for session info, 'quit' to end

The system integrates with your existing therapy database and modules.



In [9]:

import asyncio
 
async def demo_session():
     api_key = "your-gemini-api-key-here"  # Replace with actual key
     system = await start_interactive_therapy(api_key, "Interactive Demo User")
     return system
 
 # Run the interactive session
demo_system = await demo_session()

Interactive Therapy System ready!

--- Starting Interactive Session with Interactive Demo User (ID: 3) ---
Type 'quit' to end session, 'help' for commands, or just chat naturally!

AI Therapist: Okay, here we go.

Hello Interactive Demo User, welcome. It's good to have you here today. I'm really glad you've decided to take this step.

To start, I'm wondering what's been on your mind lately? What brings you in to talk today? There's no pressure to dive into anything specific right away, just whatever feels most present for you.

AI Therapist: Okay, I hear you. That sounds really difficult. Breakups are rarely easy. I imagine you're going through a lot right now.

Can you tell me a little more about the breakup? How recent was it? And how are you feeling about it? There's no need to go into extreme detail if you're not comfortable, but even a general sense of the situation would be helpful.

And beyond just the sadness that often comes with a breakup, are you experiencing any other emoti